In [1]:
from openai import OpenAI

import time
from sklearn.metrics import accuracy_score

from basicmemnet import memnet

In [2]:
client = OpenAI(api_key = 'sk---')
model = 'gpt-4o-mini'

In [2]:
md_train = memnet.DSL()
md_test = memnet.DSL()

md_train.import_gml("train_graph.gml")
md_test.import_gml("test_graph.gml")

test = md_test.get_graph()
train = md_train.get_graph()

In [3]:
len(test.nodes())

2840

In [8]:
len(test.edges())

6680

In [4]:
len(train.nodes())

11088

In [7]:
len(train.edges())

25897

In [4]:
parent_actions = ['task_1_k_cooking', 'task_2_k_cooking_with_bowls', 'task_3_k_pouring', 'task_4_k_wiping',
                  'task_5_k_cereals', 'task_6_w_hard_drive', 'task_7_w_free_hard_drive', 'task_8_w_hammering', 
                  'task_9_w_sawing']

In [5]:
examples = "\n"

for task in parent_actions:
    sub_graphs = md_train.get_stm_actions(action_attributes={"utterances": [task]})
    #print(task)
    examples = examples + f"Examples of {task}:\n"
    for gr in sub_graphs:
        #for node, data in gr.nodes(data=True):
        for u, v, link in gr.edges(data=True):
            head = gr.nodes[u]['utterances'][0]
            tail = gr.nodes[v]['utterances'][0]
            if task not in [head, tail]:
                examples = examples + f"{head} {link['link_type']} {tail}, "
                # print(head, link['link_type'], tail)
        examples = examples[:-2] + ".\n"

In [6]:
print(examples)


Examples of task_1_k_cooking:
approach has_object bowl, approach has_next hold, approach has_object bottle, approach has_next lift, place has_object bottle, place has_next retreat, retreat has_object bottle, retreat has_next approach, lift has_object bottle, lift has_next pour, retreat has_object whisk, approach has_object bowl, approach has_next hold, stir has_object whisk, stir has_next place, pour has_object bottle, pour has_next place, hold has_object bowl, hold has_next retreat, place has_object whisk, place has_next retreat, idle has_next approach, approach has_object whisk, approach has_next lift, lift has_object whisk, lift has_next stir, idle has_next approach, hold has_object bowl, hold has_next retreat, retreat has_object bowl, retreat has_object bowl, retreat has_next approach.
approach has_object bowl, approach has_next hold, hold has_object bowl, hold has_next retreat, approach has_object whisk, approach has_next lift, retreat has_object bottle, retreat has_next approach

In [7]:
instructions = """You are a knowledgeable assistant. Given a set of triples representing sub-actions, predict the most likely parent action. Your output should be one of the 9 possible parent actions: task_1_k_cooking, task_2_k_cooking_with_bowls, task_3_k_pouring, task_4_k_wiping, task_5_k_cereals, task_6_w_hard_drive, task_7_w_free_hard_drive, task_8_w_hammering, task_9_w_sawing.

Below are examples of parent actions and their corresponding sequences of sub-actions. Each triple represents a head, relation, and tail from the knowledge graph describing the parent action.\n"""

In [10]:
ground_truths = []
predictions = []
n=0
for task in parent_actions:
    sub_graphs = md_test.get_stm_actions(action_attributes={"utterances": [task]})
    for gr in sub_graphs:
        prompt = "\nWhat is the parent action of the following graph?\n"
        for u, v, link in gr.edges(data=True):
            head = gr.nodes[u]['utterances'][0]
            tail = gr.nodes[v]['utterances'][0]
            if task not in [head, tail]:
                prompt = prompt + f"{head} {link['link_type']} {tail}, "
                # print(head, link['link_type'], tail)
        prompt = prompt[:-2] + ".\n"
        prompt = prompt + "\nAnswer nothing but one of the 9 possible parent actions: task_1_k_cooking, task_2_k_cooking_with_bowls, task_3_k_pouring, task_4_k_wiping, task_5_k_cereals, task_6_w_hard_drive, task_7_w_free_hard_drive, task_8_w_hammering, task_9_w_sawing."
        ground_truths.append(task)
        
        response = client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": instructions + examples + prompt}],
                    max_tokens=10,
                    temperature=0,  # Controls the randomness of the output
                    top_p=0.9,        # Controls the diversity of the output
                    n=1,              # Number of completions to generate
                )
        answer = response.choices[0].message.content.strip()
        predictions.append(answer)
        print(n, "Truth: ", task, " Prediction: ", answer)
        n+=1
        # time.sleep(18)

0 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
1 Truth:  task_1_k_cooking  Prediction:  task_2_k_cooking_with_bowls
2 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
3 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
4 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
5 Truth:  task_1_k_cooking  Prediction:  task_4_k_wiping
6 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
7 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
8 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
9 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
10 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
11 Truth:  task_1_k_cooking  Prediction:  task_1_k_cooking
12 Truth:  task_2_k_cooking_with_bowls  Prediction:  task_2_k_cooking_with_bowls
13 Truth:  task_2_k_cooking_with_bowls  Prediction:  task_1_k_cooking
14 Truth:  task_2_k_cooking_with_bowls  Prediction:  task_4_k_wiping
15 Truth:  task_2_k_cooking_with_bowls  Prediction:  task_2_k_cooking_wi

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [12]:
print(accuracy_score(ground_truths[:-1], predictions))

0.7575757575757576
